<a href="https://colab.research.google.com/github/unaizanouman/zePOP/blob/main/zePOP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt-get update -y
!apt-get install -y mpich


Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:4 https://cli.github.com/packages stable InRelease
Get:5 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:8 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [2,153 kB]
Get:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Hit:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:12 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [6,008 kB]
Hit:13 https://ppa.launchpadcontent.net/ubuntug

In [ ]:
%%bash
cat << 'EOF' > graph_module.h
#ifndef GRAPH_MODULE_H
#define GRAPH_MODULE_H
#include <bits/stdc++.h>
using namespace std;

namespace graphmod {
    struct Graph {
        int n;
        vector<vector<int>> adj;
        vector<unordered_map<int,double>> delay;
    };

    inline Graph build_default_graph(int n){
        Graph g;
        g.n=n;
        g.adj.resize(n);
        g.delay.resize(n);
        mt19937_64 rng(12345);

        for(int i=0;i<n;i++){
            int a=(i+1)%n, b=(i-1+n)%n;
            g.adj[i].push_back(a);
            g.adj[i].push_back(b);
        }

        for(int i=0;i<n;i++){
            int extra=(i*7+13)%n;
            if(extra!=i){
                g.adj[i].push_back(extra);
                g.adj[extra].push_back(i);
            }
        }

        for(int i=0;i<n;i++){
            sort(g.adj[i].begin(), g.adj[i].end());
            g.adj[i].erase(unique(g.adj[i].begin(), g.adj[i].end()), g.adj[i].end());
        }

        for(int u=0;u<n;u++){
            for(int v:g.adj[u]){
                if(!g.delay[u].count(v)){
                    rng.seed(5000 + min(u,v)*10000 + max(u,v));
                    double d = 1.0 + (rng()%9000)/1000.0;
                    g.delay[u][v]=d;
                    g.delay[v][u]=d;
                }
            }
        }
        return g;
    }
}
#endif
EOF

cat << 'EOF' > zepop_library.h
#ifndef ZEPOP_LIBRARY_H
#define ZEPOP_LIBRARY_H
#include <mpi.h>
#include <bits/stdc++.h>
#include "graph_module.h"
using namespace std;
using namespace graphmod;

namespace zepop {
    const double INF = 1e18;

    inline vector<double> compute_distances(const Graph &G,int rank,int n){
        vector<double> dist(n,INF);
        dist[rank]=0;

        for(auto &p:G.delay[rank]) dist[p.first]=p.second;

        for(int r=0;r<n+5;r++){
            vector<double> sendbuf=dist;
            for(int nb:G.adj[rank])
                MPI_Send(sendbuf.data(),n,MPI_DOUBLE,nb,0,MPI_COMM_WORLD);

            bool changed=false;
            for(int nb:G.adj[rank]){
                vector<double> recv(n);
                MPI_Recv(recv.data(),n,MPI_DOUBLE,nb,0,MPI_COMM_WORLD,MPI_STATUS_IGNORE);
                double w=G.delay[rank].at(nb);
                for(int k=0;k<n;k++){
                    double cand=w+recv[k];
                    if(cand<dist[k]){
                        dist[k]=cand;
                        changed=true;
                    }
                }
            }
            int l=changed?1:0, g=0;
            MPI_Allreduce(&l,&g,1,MPI_INT,MPI_SUM,MPI_COMM_WORLD);
            if(g==0) break;
        }
        return dist;
    }

    inline vector<double> gather_all_dist(const vector<double>&dist,int n){
        vector<double> all(n*n);
        MPI_Allgather(dist.data(),n,MPI_DOUBLE,all.data(),n,MPI_DOUBLE,MPI_COMM_WORLD);
        return all;
    }

    inline int elect_leader(const vector<double>&C){
        double best=-1;
        int leader=-1;
        for(int i=0;i<C.size();i++){
            if(C[i]>best){
                best=C[i];
                leader=i;
            }
        }
        return leader;
    }
}
#endif
EOF

cat << 'EOF' > main_zepop_code.cpp
#include <mpi.h>
#include <bits/stdc++.h>
#include "graph_module.h"
#include "zepop_library.h"
using namespace std;
using namespace graphmod;
using namespace zepop;

int main(int argc,char**argv){
    MPI_Init(&argc,&argv);
    int rank,size;
    MPI_Comm_rank(MPI_COMM_WORLD,&rank);
    MPI_Comm_size(MPI_COMM_WORLD,&size);

    Graph G = build_default_graph(size);

    vector<double> dist = compute_distances(G,rank,size);
    vector<double> allD = gather_all_dist(dist,size);

    auto D=[&](int u,int v){ return allD[u*size+v]; };

    double Tx=0;
    for(int i=0;i<size;i++) if(i!=rank) Tx+=D(rank,i);
    double closeness=(size>1)?(double)(size-1)/Tx : 0;

    vector<double> allC(size);
    MPI_Allgather(&closeness,1,MPI_DOUBLE,allC.data(),1,MPI_DOUBLE,MPI_COMM_WORLD);

    int leader = elect_leader(allC);

    int parent=-1;
    if(rank!=leader){
        double best=D(rank,leader);
        int choice=-1;
        for(int nb:G.adj[rank]){
            double w=G.delay[rank].at(nb);
            if(fabs((w + D(nb,leader)) - best) < 1e-6){
                if(choice==-1 || nb<choice) choice=nb;
            }
        }
        parent=choice;
    }

    vector<int> allP(size);
    MPI_Allgather(&parent,1,MPI_INT,allP.data(),1,MPI_INT,MPI_COMM_WORLD);

    if(rank==0){
        cout<<"=== ZePoP Clean Modular Output ===\n";
        cout<<"Leader: "<<leader<<"\n";
        for(int i=0;i<size;i++){
            cout<<"Node "<<i<<" -> Parent: "<<allP[i]<<"\n";
        }
    }

    MPI_Finalize();
    return 0;
}
EOF


In [ ]:
!mpicxx -std=c++11 -O2 main_zepop_code.cpp -o zepop


In [ ]:
!mpirun --allow-run-as-root --oversubscribe -np 8 ./zepop


=== ZePoP Clean Modular Output ===
Leader: 0
Node 0 -> Parent: -1
Node 1 -> Parent: 0
Node 2 -> Parent: 1
Node 3 -> Parent: 4
Node 4 -> Parent: 5
Node 5 -> Parent: 0
Node 6 -> Parent: 7
Node 7 -> Parent: 0
